In [1]:
import tensorflow as tf
import numpy as np

fd = open('./Desktop/mnist/train-images.idx3-ubyte')
images = np.fromfile(file=fd, dtype=np.uint8)
images = images[16:].reshape([60000,28,28]).astype(np.float)
images = images/127 - 1.0

fd = open('./Desktop/mnist/train-labels.idx1-ubyte')
labels = np.fromfile(file=fd, dtype=np.uint8)
labels = labels[8:].reshape([60000]).astype(np.int)


In [2]:
fd = open('./Desktop/mnist/t10k-images.idx3-ubyte')
images2 = np.fromfile(file=fd, dtype=np.uint8)
images2 = images2[16:].reshape([10000,28,28]).astype(np.float)
images2 = images2/127 - 1.0

fd = open('./Desktop/mnist/t10k-labels.idx1-ubyte')
labels2 = np.fromfile(file=fd, dtype=np.uint8)
labels2 = labels2[8:].reshape([10000]).astype(np.int)

In [3]:
type(images)
print(images.shape)
print(labels.shape)

(60000, 28, 28)
(60000,)


In [4]:
X = tf.placeholder(shape=[None, 28*28], dtype=tf.float32, name='X')
Y = tf.placeholder(shape=[None], dtype=tf.int64, name='Y')
is_training=tf.placeholder(tf.bool)

def fully_connected(x, input_num, output_num, name):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([input_num, output_num], stddev=0.02, name='w'))
        b = tf.Variable(tf.zeros([output_num]), name='b')
        return tf.matmul(x, w) + b

# hidden1
#weights1 = tf.Variable(tf.truncated_normal([28*28, 300], stddev=0.02), name="weights1")
#biases1 = tf.Variable(tf.zeros([300]), name="biases1")
hidden1 = fully_connected(X, 28*28, 300, "weights1") 
hidden1 = tf.nn.relu(hidden1, "relu1")
hidden1 = tf.layers.dropout(hidden1, rate=0.5, training=is_training)

# hidden2
#weights2 = tf.Variable(tf.truncated_normal([300, 200], stddev=0.02), name="weights2")
#biases2 = tf.Variable(tf.zeros([200]), name="biases2")
hidden2 = fully_connected(hidden1, 300, 200, "weights2")
hidden2 = tf.nn.relu(hidden2, "relu2")
hidden2 = tf.layers.dropout(hidden2, rate=0.5, training=is_training)

# ouput
weights3 = tf.Variable(tf.truncated_normal([200, 10]),name="weights3")
biases3 = tf.Variable(tf.truncated_normal([10]), name="biases3")

Y_= tf.matmul(hidden2,weights3) + biases3

Y_onehot = tf.one_hot(Y, 10, axis=1)

softmax = tf.nn.softmax(Y_)

correct = tf.equal(tf.argmax(softmax,1), tf.argmax(Y_onehot,1))
acc = tf.reduce_mean(tf.cast(correct, tf.float32))

#Creat Model
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Y_, labels=Y_onehot))

trainer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
optimize = trainer.minimize(loss)

# In training loop

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    batch_size= 1000
    batch_count = 60000 // batch_size   
    
    for epoch in range(100):
        total_loss = 0
        for i in range(batch_count):
            batch_index = i * batch_size
            img = np.reshape(images[batch_index:batch_index+batch_size], [batch_size, 28*28])
            label = labels[batch_index:batch_index+batch_size]
            _, loss_v = sess.run([optimize, loss],feed_dict={X:img, Y:label,is_training:True})
            total_loss += loss_v 
            
        print(total_loss)
        
    TRAINING_ACC = sess.run(acc, feed_dict={X:img, Y:label, is_training:False})
    print(TRAINING_ACC)
    
    TEST_ACCURACY_ACC = sess.run(acc, feed_dict={X:np.reshape(images2,[10000,28*28]), Y:labels2, is_training:False})
    print(TEST_ACCURACY_ACC)


126.898569703
107.503109694
89.0488408804
64.89900738
48.9146120548
42.4128498733
38.2527349293
34.9397937655
32.5519328713
31.0539443195
29.1793033779
28.2638496161
27.001660198
25.8469117284
25.4806106091
24.2826193571
23.7050693035
23.2762051523
22.7891983092
22.161026448
21.6709611565
21.2367541194
20.7980094552
20.6529424787
20.1743164361
19.8021818995
19.320833385
19.0914211124
18.6053534895
18.5232954025
18.1887457669
17.8946870565
17.6684270948
17.2950349748
17.2407151759
16.9292016476
16.7087961435
16.379143551
16.4391336441
16.1817567647
16.0046537519
15.8534002751
15.6123029888
15.2578205466
15.0158302784
15.0286996514
15.0335495621
14.7559705526
14.4932285994
14.2954267412
14.1002798378
14.1274273992
14.0317626745
13.8045534045
13.62009193
13.4847875535
13.7083491832
13.3698154092
12.9632617831
13.2463814616
12.8662103266
12.8669047356
12.6227111816
12.4088551402
12.5421205088
12.4774144888
12.3458062261
12.1012795493
12.2308906689
11.9343981147
12.2029470503
11.835827738
1